<a href="https://colab.research.google.com/github/NBK-code/TextWorld_Game_GRPO/blob/main/TextWorld_Game_Ver_3_3_A_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data collection changed to align with stepwise optimization.

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
!pip install -q textworld
import textworld
import json
from pathlib import Path
from unsloth import FastLanguageModel
import torch.nn.functional as F
import torch
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 9.1 MB/s eta 0:00:00
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-27 16:45:39 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-27 16:45:40 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
max_seq_length = 1024  # Can increase for longer reasoning traces
lora_rank = 8  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6
)

In [ ]:
def count_trainable_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Total parameters:     {total_params:,}")
    print(f"Percentage trainable: {100 * trainable_params / total_params:.4f}%")

In [ ]:
count_trainable_parameters(model)

Trainable parameters: 0
Total parameters:     4,628,680,704
Percentage trainable: 0.0000%


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,)

Unsloth 2025.7.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
count_trainable_parameters(model)

Trainable parameters: 20,971,520
Total parameters:     4,649,652,224
Percentage trainable: 0.4510%


0.4510%

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
    

In [ ]:
system_message = """
You are playing a text-based adventure game in a world with various locations and items. Your goal is to explore the world and collect the hidden coin. You can move in different directions (e.g., north, south, east, west) and interact with objects and characters.

### Game Rules:
- You can move using: "go north", "go south", "go east", "go west".
- When a coin is present, it will be clearly described.
- Use "take coin" to collect it.
- Do not return to rooms you've already visited unless necessary.

### Objective:
Your goal is to find the coin present somewhere in the environment. When you find the coin, you can claim it by using the command "take coin"

### Response Format:
<think> your reasoning goes here </think>
<command> your next action goes here </command>

Do not use any other format. Follow this structure exactly.

### Example 1:
Map So Far:
Hallway: east → Kitchen
Kitchen: west → Hallway

Last Move:
From Hallway → go east → Kitchen

Current State:
-= Kitchen =-
You are in the Kitchen. There is no coin here. There are various utensils.

You need an unblocked exit? You should try going west. You don't like doors? Why not try going north, that entranceway is unblocked.

<think>I'm in the Kitchen. Using the "Last Move" and the "Map So Far", I see that I took east from Hallway to arrive here. If I go west I will just go back to the Hallway. But there was no coin in the Hallway. So I won't go back to the Hallway. I will try to explore other options. I see that I can go north. So I should take it.</think>
<command>go north</command>"""

In [ ]:
def extract_initial_game_state(text):
    """
    Extracts the initial room description and its name from the game start text.
    """
    match = re.search(r'(-= .*? =-.*?)(?=>)', text, re.DOTALL)
    if not match:
        return None, None

    description = match.group(1)
    cleaned = re.sub(r'\n\s*\n+', '\n\n', description.strip())

    # Extract room name from -= Room =-
    room_match = re.search(r"-= (.*?) =-", cleaned)
    room_name = room_match.group(1).strip() if room_match else None

    return cleaned, room_name

In [ ]:
def extract_thought_and_command(text):
    """
    Extracts the first <think>...</think> block and <command>...</command> block from the given text.
    Returns a tuple: (thought, command), where either can be None if not found.
    """
    think_match = re.search(r"<think>(.*?)</think>", text, re.DOTALL)
    command_match = re.search(r"<command>(.*?)</command>", text, re.DOTALL)

    thought = think_match.group(1).strip() if think_match else None
    command = command_match.group(1).strip() if command_match else None

    return thought, command

In [ ]:
def parse_game_state(raw_text):
    score_match = re.search(r"<score>\s*(\d+)\s*</score>", raw_text)
    score = int(score_match.group(1)) if score_match else 0 #fallback to 0

    moves_match = re.search(r"<moves>\s*(\d+)\s*</moves>", raw_text)
    moves = int(moves_match.group(1)) if moves_match else 0

    text = raw_text.split("<score>")[0].strip()
    state_description = re.sub(r'\n\s*\n+', '\n\n', text.strip())

    return state_description, score, moves

In [ ]:
def take_action(action: str):

    state = env.step(action)
    state_description, score, moves = parse_game_state(state[0]['raw'])

    # Extract room name from state_description
    room_match = re.search(r"-= (.*?) =-", state_description)
    room_name = room_match.group(1).strip() if room_match else None

    return state_description, score, moves, room_name

In [ ]:
def format_room_graph(room_graph):
    lines = ["### Map So Far:"]
    for room, connections in room_graph.items():
        edges = [f"{dir} → {dest}" for dir, dest in connections.items()]
        lines.append(f"{room}: {', '.join(edges)}")
    return "\n".join(lines)

In [ ]:
def compute_completion_logprob_from_ids(full_input_ids, prompt_len, model):
    """
    computes log-prob of tokens at i+1 given logits at i.
    """
    attention_mask = torch.ones_like(full_input_ids)

    logits = model(full_input_ids, attention_mask=attention_mask).logits  # shape (1, T, vocab)
    log_probs = torch.nn.functional.log_softmax(logits[:, :-1, :], dim=-1)  # shape (1, T-1, vocab)

    target_ids = full_input_ids[:, 1:]  # shape (1, T-1)
    token_log_probs = log_probs.gather(2, target_ids.unsqueeze(-1)).squeeze(-1)  # shape (1, T-1)
    token_log_probs = token_log_probs.squeeze(0)  # shape (T-1,)

    completion_mask = torch.zeros_like(token_log_probs)
    completion_mask[prompt_len - 1:] = 1.0  # offset by 1 due to shifting

    completion_logprob = (token_log_probs * completion_mask).sum()

    return completion_logprob, token_log_probs, completion_mask

In [ ]:
def run_episode(env, model, tokenizer, system_message, max_steps=20):
    state = env.reset()
    initial_game_state, prev_room = extract_initial_game_state(state.feedback)
    user_message = initial_game_state

    visited_room_names = set()
    visited_room_names.add(prev_room)
    room_graph = {}

    logprobs = []
    tokenwise_log_probs = []
    moves = 0
    score = 0

    episode_full_input_ids = []
    episode_prompt_len = []


    for step in range(1, max_steps + 1):
        prompt = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ],
            tokenize=False,
            add_generation_prompt=True)

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.2,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )

        response = tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

        #print(f"🤖 Response: {response}")

        # Get prompt length and full sequence
        prompt_len = inputs["input_ids"].shape[1]
        full_input_ids = outputs[0].unsqueeze(0)  # shape (1, T)

        # Compute log-probs on completion portion
        with torch.no_grad():
            completion_logprob, token_log_probs, completion_mask = compute_completion_logprob_from_ids(
                                                  full_input_ids, prompt_len, model)

        episode_full_input_ids.append(full_input_ids)
        episode_prompt_len.append(prompt_len)
        logprobs.append(completion_logprob)
        tokenwise_log_probs.append(token_log_probs)

        thought, action = extract_thought_and_command(response)
        if not action:
            print("\n No valid command generated. Ending episode.")
            step = max_steps
            break

        state, maybe_score, moves, next_room = take_action(action)

        if maybe_score is not None:
            score = maybe_score

        if score == 1:
            print("Coin Found!")
            break

        # Map tracking
        if next_room and next_room not in visited_room_names:
            visited_room_names.add(next_room)

        normalized_action = action.lower()
        if prev_room and next_room and normalized_action.startswith("go "):
            direction = normalized_action.split()[1]
            room_graph.setdefault(prev_room, {})[direction] = next_room
            reverse = {"north": "south", "south": "north", "east": "west", "west": "east"}
            rev_dir = reverse.get(direction)
            if rev_dir:
                room_graph.setdefault(next_room, {})[rev_dir] = prev_room

        graph_summary = format_room_graph(room_graph)
        last_move_description = f"From {prev_room} → {action.lower()} → {next_room}"
        user_message = (
            f"{graph_summary}\n\n"
            f"### Last Move:\n{last_move_description}\n\n"
            f"### Current State:\n{state}\n\n"
            f"---\n\n"
            f"### Please respond strictly in the following format:\n\n"
            f"<think>...</think>\n<command>...</command>"
        )

        prev_room = next_room

    reward = score - 0.01 * step
    return logprobs, tokenwise_log_probs, reward, episode_full_input_ids, episode_prompt_len

In [ ]:
def collect_episodes(env, model, tokenizer, system_message, N=8):
    all_logprobs = []
    all_tokenwise_log_probs = []
    all_rewards = []
    all_full_input_ids = []
    all_prompt_len = []

    for i in range(N):
        print(f"\n🎮 Episode {i + 1} / {N}")
        logprobs, tokenwise_log_probs, reward, full_input_ids, prompt_len = run_episode(env, model, tokenizer, system_message)
        all_logprobs.append(logprobs)
        all_tokenwise_log_probs.append(tokenwise_log_probs)
        all_rewards.append(reward)
        all_full_input_ids.append(full_input_ids)
        all_prompt_len.append(prompt_len)
        print(f"✅ Reward: {reward:.4f} | Steps: {len(logprobs)}")

    return all_logprobs, all_tokenwise_log_probs, all_rewards, all_full_input_ids, all_prompt_len

In [ ]:
def compute_advantages(all_rewards):

    rewards = torch.tensor(all_rewards, dtype=torch.float32)
    mean_r = rewards.mean()
    std_r = rewards.std(unbiased=False) + 1e-8  # unbiased=True by default
    advantages = (rewards - mean_r) / std_r
    return advantages.tolist(), mean_r

In [ ]:
# ------------------ Configs ------------------
levels = [5, 6, 7]
seeds = [10, 14, 18, 22, 26, 30, 34, 38, 42, 46]  # 10 seeds
episodes_per_game = 8

save_dir = Path("grpo_dataset")
save_dir.mkdir(exist_ok=True)
save_path = save_dir / "grpo_data.jsonl"

In [ ]:
with open(save_path, "w") as f:
    for level in levels:
        for seed in seeds:
            gamefile = f"coin_level{level}_{seed}.z8"
            !tw-make tw-coin_collector --level {level} --seed {seed} --output {gamefile} --silent
            env = textworld.start(gamefile)
            print(f"\n----- 🌍 Level: {level} | Seed: {seed} -----")

            # Get N rollouts for this environment
            all_logprobs, all_tokenwise_log_probs, all_rewards, all_full_input_ids, all_prompt_len = collect_episodes(
                env, model, tokenizer, system_message, episodes_per_game)

            # Compute advantage across all episodes
            all_advantages, mean_reward = compute_advantages(all_rewards)  # same length as episodes_per_game

            # Save one JSONL line per step
            for i in range(episodes_per_game):
                for step in range(len(all_full_input_ids[i])):
                    example = {
                        "level": level,
                        "seed": seed,
                        "episode": i,
                        "step": step,
                        "input_ids": all_full_input_ids[i][step].squeeze(0).tolist(),
                        "prompt_len": all_prompt_len[i][step],
                        "logprob_old": all_logprobs[i][step].item(),
                        "tokenwise_logprobs": all_tokenwise_log_probs[i][step].tolist(),
                        "advantage": all_advantages[i],
                        "reward": all_rewards[i],
                        "mean_reward": mean_reward.item(),
                    }
                    f.write(json.dumps(example) + "\n")

            print(f"\n🌍 Level: {level} | Seed: {seed} | Mean reward: {mean_reward:.4f}")


print(f"\n✅ All data saved to: {save_path}")


----- 🌍 Level: 5 | Seed: 10 -----

🎮 Episode 1 / 8
Coin Found!
✅ Reward: 0.8100 | Steps: 19

🎮 Episode 2 / 8
Coin Found!
✅ Reward: 0.9300 | Steps: 7

🎮 Episode 3 / 8
✅ Reward: -0.2000 | Steps: 20

🎮 Episode 4 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 5 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 6 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 7 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 8 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🌍 Level: 5 | Seed: 10 | Mean reward: 0.7862

----- 🌍 Level: 5 | Seed: 14 -----

🎮 Episode 1 / 8
Coin Found!
✅ Reward: 0.9300 | Steps: 7

🎮 Episode 2 / 8
Coin Found!
✅ Reward: 0.8800 | Steps: 12

🎮 Episode 3 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 4 / 8
Coin Found!
✅ Reward: 0.9500 | Steps: 5

🎮 Episode 5 / 8
Coin Found!
✅ Reward: 0.8700 | Steps: 13

🎮 Episode 6 / 8
Coin Found!
✅ Reward: 0.8500 | Steps: 15

🎮 Episode 7 / 8
Coin Found!
✅ Reward: 0.8600 | Steps: 14

🎮 Episode 8 / 8
Coin Found!
✅ Rewa

In [ ]:
import pandas as pd
file_path = "/content/grpo_dataset/grpo_data.jsonl"
df = pd.read_json(file_path, lines=True)
df.head()

,level,seed,episode,step,input_ids,prompt_len,logprob_old,tokenwise_logprobs,advantage,reward,mean_reward
0,5,10,0,0,"[128000, 128000, 128006, 9125, 128007, 271, 38...",452,-29.546875,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.06325,0.81,0.78625
1,5,10,0,1,"[128000, 128000, 128006, 9125, 128007, 271, 38...",526,-36.250000,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.06325,0.81,0.78625
2,5,10,0,2,"[128000, 128000, 128006, 9125, 128007, 271, 38...",514,-49.125000,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.06325,0.81,0.78625
3,5,10,0,3,"[128000, 128000, 128006, 9125, 128007, 271, 38...",485,-47.437500,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.06325,0.81,0.78625
4,5,10,0,4,"[128000, 128000, 128006, 9125, 128007, 271, 38...",484,-33.718750,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.06325,0.81,0.78625


In [ ]:
df

,level,seed,episode,step,input_ids,prompt_len,logprob_old,tokenwise_logprobs,advantage,reward,mean_reward
0,5,10,0,0,"[128000, 128000, 128006, 9125, 128007, 271, 38...",452,-29.546875,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.063250,0.81,0.78625
1,5,10,0,1,"[128000, 128000, 128006, 9125, 128007, 271, 38...",526,-36.250000,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.063250,0.81,0.78625
2,5,10,0,2,"[128000, 128000, 128006, 9125, 128007, 271, 38...",514,-49.125000,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.063250,0.81,0.78625
3,5,10,0,3,"[128000, 128000, 128006, 9125, 128007, 271, 38...",485,-47.437500,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.063250,0.81,0.78625
4,5,10,0,4,"[128000, 128000, 128006, 9125, 128007, 271, 38...",484,-33.718750,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.063250,0.81,0.78625
...,...,...,...,...,...,...,...,...,...,...,...
3073,7,46,7,3,"[128000, 128000, 128006, 9125, 128007, 271, 38...",537,-45.843750,"[-13.703125, -4.18359375, -22.34375, -12.65625...",0.556319,0.92,0.90125
3074,7,46,7,4,"[128000, 128000, 128006, 9125, 128007, 271, 38...",506,-59.531250,"[-13.6953125, -4.18359375, -22.375, -12.671875...",0.556319,0.92,0.90125
3075,7,46,7,5,"[128000, 128000, 128006, 9125, 128007, 271, 38...",556,-34.718750,"[-13.703125, -4.18359375, -22.34375, -12.65625...",0.556319,0.92,0.90125
3076,7,46,7,6,"[128000, 128000, 128006, 9125, 128007, 271, 38...",570,-46.031250,"[-13.703125, -4.18359375, -22.34375, -12.65625...",0.556319,0.92,0.90125
